In [19]:
!nvidia-smi
!pip install tabulate
!pip install tensorboard

Tue Oct 17 18:40:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.199.02   Driver Version: 470.199.02   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A10          Off  | 00000000:17:00.0 Off |                    0 |
|  0%   46C    P0    59W / 150W |   5089MiB / 22731MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A10          Off  | 00000000:31:00.0 Off |                    0 |
|  0%   

In [20]:
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import random_split, DataLoader,TensorDataset
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
import random
from datetime import time
from dateutil.parser import parse
import os
import numpy as np
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from tabulate import tabulate
import re
import multiprocessing as mp
import math
import torch.optim as optim
import pickle
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.tensorboard import SummaryWriter
from sklearn.preprocessing import MinMaxScaler
from torch.optim.lr_scheduler import StepLR

In [21]:
def preprocess_data(file_path):
    df = pd.read_excel(file_path)
    df.columns = ['CITIZENID', 'WILAYAT_CODE','TIMESTAMP', 'AGE', 'SEX', 'VOTE_REG_YR','REGIONID']
    df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
    df['AGE'] = pd.to_numeric(df['AGE'], errors='coerce')
    custom_column_names = ["Time","CITIZENID","REGION_ID","WILAYAT_CODE","Male","Female","20-30","30-40",
                           "40-50","50-60","60-70","70-80","80+"]

    new_df = pd.DataFrame(columns=custom_column_names)
    new_df['Time'] = df['TIMESTAMP'].dt.strftime('%H:%M')
    new_df['CITIZENID'] = df['CITIZENID']
    new_df['REGION_ID'] = df['REGIONID']
    
    new_df['WILAYAT_CODE'] = df['WILAYAT_CODE']
    new_df['Male'] = (df['SEX'] == 1).astype(int)
    new_df['Female'] = (df['SEX'] != 1).astype(int)
    labels = ["20-30", "30-40", "40-50", "50-60", "60-70", "70-80", "80+"]

    for label in labels:
        if label != "80+":
            age_group_mask = (df['AGE'] >= int(label.split('-')[0])) & (df['AGE'] < int(label.split('-')[1]))
        else:
            age_group_mask = df['AGE'] >= 80
        new_df[label] = age_group_mask.astype(int)

    new_df.sort_values(by=['WILAYAT_CODE', 'Time'], inplace=True)
    return new_df

In [22]:
file_paths = ['/home/aiadmin1/Desktop/ND/edata_2022.xlsx','/home/aiadmin1/Desktop/ND/edata_2012.xlsx','/home/aiadmin1/Desktop/ND/edata_2015.xlsx',
               '/home/aiadmin1/Desktop/ND/edata_2016.xlsx','/home/aiadmin1/Desktop/ND/edata_2019.xlsx','/home/aiadmin1/Desktop/ND/edata_2011.xlsx']
data_frames = [preprocess_data(file_path) for file_path in file_paths]
wilayat_data_frames = {}
start_time = '07:00'
end_time = '18:59'
all_times = [f"{hour:02d}:{minute:02d}" for hour in range(7, 19) for minute in range(0, 60)]

In [23]:
batch_size = 540
epochs = 5000
window_size = 60
test_size = 0.1

In [24]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=60):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(1)].squeeze(1).unsqueeze(0)
        return self.dropout(x)

class TransformerPredictor(nn.Module):
    def __init__(self, input_dim, d_model, nhead, num_encoder_layers, output_dim):
        super(TransformerPredictor, self).__init__()
        self.embedding = nn.Linear(input_dim, d_model)
        self.positional_encoding = PositionalEncoding(d_model)
        self.transformer = nn.Transformer(d_model, nhead, num_encoder_layers)
        self.fc = nn.Linear(d_model, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        x = self.positional_encoding(x)
        x = self.transformer.encoder(x)
        x = self.fc(x)  
        return x

In [30]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

model = TransformerPredictor(input_dim=9, d_model=128, nhead=16, num_encoder_layers=6, output_dim=9)
params_in_millions = count_parameters(model) / 1e6
print(f'The model has {params_in_millions:.2f} million trainable parameters.')

The model has 7.52 million trainable parameters.


In [31]:
def outlier_replacement(df, wilayat_id):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    df_out = df.copy()
    df_out[((df_out < (Q1 - 1.5 * IQR)) | (df_out > (Q3 + 1.5 * IQR)))] = 0
    
    scaler_path = f"/home/aiadmin1/Desktop/OMAN_AI_PROJECT/minmax/scaler_{wilayat_id}.pkl"
    if os.path.exists(scaler_path):
        with open(scaler_path, "rb") as f:
            scaler = pickle.load(f)
        df_normalized = scaler.transform(df_out)
    else:
        scaler = MinMaxScaler()
        df_normalized = scaler.fit_transform(df_out)
        with open(scaler_path, "wb") as f:
            pickle.dump(scaler, f)
        
    model_path = f'/home/aiadmin1/Desktop/OMAN_AI_PROJECT/Transformer44M_2/model_{wilayat_id}.pt'
    print(f'model_{wilayat_id} is training for dataset of {dataset_i}')

    if os.path.exists(model_path):
        model = torch.load(model_path)
        print(f"Model for {wilayat_id} loaded from saved state.")
    else:
        model = TransformerPredictor(input_dim=9, d_model=128, nhead=16, num_encoder_layers=6, output_dim=9)
        print(f"Created new model for {wilayat_id}.")
        model = nn.DataParallel(model)

    wilayat_models[wilayat_id] = model
    future_pred_window = 60
    input_sequences = []
    target_sequences = []

    for i in range(0, len(df_normalized) - 60- 60 + 1):
        input_data = df_normalized[i: i + 60]
        target_data = df_normalized[i + 60 : i + 60+ 60]
        input_sequences.append(input_data)
        target_sequences.append(target_data)
        
    input_sequences = np.array(input_sequences)
    target_sequences = np.array(target_sequences)
    input_tensor = torch.Tensor(input_sequences)
    target_tensor = torch.Tensor(target_sequences)
    
    return input_tensor ,target_tensor , scaler

In [ ]:
wilayat_models = {}
for dataset_i, df in enumerate(data_frames):
    df_time_range = df[(df['Time'] >= start_time) & (df['Time'] <= end_time)]
    
    for wilayat_id in df_time_range['WILAYAT_CODE'].unique():
        
        wilayat_data_frames[wilayat_id] = df_time_range[df_time_range['WILAYAT_CODE'] == wilayat_id].copy()
        wilayat_data_frames[wilayat_id].set_index('Time', inplace=True)
        wilayat_data_frames[wilayat_id].drop(columns=['WILAYAT_CODE', 'REGION_ID', 'CITIZENID'], inplace=True)
        wilayat_data_frames[wilayat_id] = wilayat_data_frames[wilayat_id].groupby('Time').sum()
        wilayat_data_frames[wilayat_id] = wilayat_data_frames[wilayat_id].reindex(all_times)
        wilayat_data_frames[wilayat_id].fillna(0, inplace=True)

    for wilayat_id, wilayat_df in wilayat_data_frames.items():
        
        input_sequences,target_sequences,scaler = outlier_replacement(wilayat_df, wilayat_id)
        X_train, X_test, y_train, y_test = train_test_split(input_sequences, target_sequences, test_size=test_size, shuffle=False)
        print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        train_inputs = torch.Tensor(X_train).to(device)
        train_targets = torch.Tensor(y_train).to(device)
        test_inputs = torch.Tensor(X_test).to(device)
        test_targets = torch.Tensor(y_test).to(device)
        model = wilayat_models[wilayat_id].to(device)

        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=0.00002)

        train_dataset = TensorDataset(train_inputs, train_targets)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_dataset = TensorDataset(test_inputs, test_targets)
        test_loader = DataLoader(test_dataset, batch_size=batch_size)
        writer = SummaryWriter(f"runs3/WilayatID_{wilayat_id}")

        for epoch in range(epochs):
            model.train()
            for inputs, targets in train_loader:

                inputs = inputs.to(device)
                targets = targets.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward() 
                optimizer.step()
                writer.add_scalar('Training loss', loss.item(), epoch)

            if epoch %1000 == 0:        
                model.eval()
                with torch.no_grad():
                    for inputs, targets in test_loader:
                        inputs = inputs.to(device)
                        targets = targets.to(device)
                        test_outputs = model(inputs)
                        mse = criterion(test_outputs, targets).item()
                        writer.add_scalar('Testing loss', mse, epoch)
                        test_outputs_np = test_outputs.cpu().numpy()  
                        reshaped_outputs = test_outputs_np.reshape(-1,test_outputs_np.shape[-2] ,test_outputs_np.shape[-1])
                        reshaped_outputs = np.sum(reshaped_outputs , axis=1)
                        original_scale_outputs = scaler.inverse_transform(reshaped_outputs)
                        original_scale_outputs_rounded = np.round(original_scale_outputs).astype(int)
                        print(original_scale_outputs_rounded[-1])

                torch.save(model, f'/home/aiadmin1/Desktop/OMAN_AI_PROJECT/Transformer44M_2/model_{wilayat_id}.pt')
                print('Training loss', loss.item(), 'Testing loss', mse, epoch, f"model_{wilayat_id} model saved")

        writer.close()

model_1 is training for dataset of 0
Model for 1 loaded from saved state.
torch.Size([540, 60, 9]) torch.Size([61, 60, 9]) torch.Size([540, 60, 9]) torch.Size([61, 60, 9])
[131  99  30  66  53  27   2   0   0]
Training loss 0.07858190685510635 Testing loss 0.09208472818136215 0 model_1 model saved
[131 122  18  51  87  34  25   0   0]
Training loss 0.027131086215376854 Testing loss 0.07069817185401917 1000 model_1 model saved
[133 122  17  53  87  35  25   0   0]
Training loss 0.02278168313205242 Testing loss 0.06873799115419388 2000 model_1 model saved
[133 124  16  53  86  36  25   0   0]
Training loss 0.021267706528306007 Testing loss 0.06874633580446243 3000 model_1 model saved
[132 123  16  53  85  36  24   0   0]
Training loss 0.020471543073654175 Testing loss 0.068888358771801 4000 model_1 model saved
model_2 is training for dataset of 0
Model for 2 loaded from saved state.
torch.Size([540, 60, 9]) torch.Size([61, 60, 9]) torch.Size([540, 60, 9]) torch.Size([61, 60, 9])
[259 177

model_11 is training for dataset of 0
Model for 11 loaded from saved state.
torch.Size([540, 60, 9]) torch.Size([61, 60, 9]) torch.Size([540, 60, 9]) torch.Size([61, 60, 9])
[533 393 227 305 164 124  57  28   0]
Training loss 0.06882286816835403 Testing loss 0.06447303295135498 0 model_11 model saved
[517 484 161 285 283 139 108  26   0]
Training loss 0.009712216444313526 Testing loss 0.04824761301279068 1000 model_11 model saved
[522 488 157 286 284 144 106  26   0]
Training loss 0.0054942406713962555 Testing loss 0.041791368275880814 2000 model_11 model saved
[517 487 154 288 280 146 105  27   0]
Training loss 0.003825712949037552 Testing loss 0.03896569833159447 3000 model_11 model saved
[516 487 154 288 280 148 104  27   0]
Training loss 0.002889123745262623 Testing loss 0.03755023702979088 4000 model_11 model saved
model_12 is training for dataset of 0
Model for 12 loaded from saved state.
torch.Size([540, 60, 9]) torch.Size([61, 60, 9]) torch.Size([540, 60, 9]) torch.Size([61, 60

[63 21 20 27  0  0  0  0  0]
Training loss 0.07838710397481918 Testing loss 0.07451744377613068 0 model_21 model saved
[43 23  0 23 23  0  0  0  0]
Training loss 0.04204137623310089 Testing loss 0.045030154287815094 1000 model_21 model saved
[44 23  0 23 23  0  0  0  0]
Training loss 0.04135669022798538 Testing loss 0.0453784354031086 2000 model_21 model saved
[44 23  0 23 23  0  0  0  0]
Training loss 0.04112822934985161 Testing loss 0.045621953904628754 3000 model_21 model saved
[44 23  0 23 24  0  0  0  0]
Training loss 0.04105876758694649 Testing loss 0.04583841934800148 4000 model_21 model saved
model_22 is training for dataset of 0
Model for 22 loaded from saved state.
torch.Size([540, 60, 9]) torch.Size([61, 60, 9]) torch.Size([540, 60, 9]) torch.Size([61, 60, 9])
[28  0  0  0  0  0  0  0  0]
Training loss 0.010534708388149738 Testing loss 0.008961292915046215 0 model_22 model saved
[27  0  0  0  0  0  0  0  0]
Training loss 0.010374060831964016 Testing loss 0.009010070003569126

[82 58 19 48 30 22  0  0  0]
Training loss 0.01973761059343815 Testing loss 0.04747142642736435 1000 model_31 model saved
[81 56 18 46 30 22  0  0  0]
Training loss 0.01757834292948246 Testing loss 0.049786943942308426 2000 model_31 model saved
[78 56 18 45 30 21  0  0  0]
Training loss 0.01679401844739914 Testing loss 0.05059259384870529 3000 model_31 model saved
[78 56 18 44 30 21  0  0  0]
Training loss 0.016240978613495827 Testing loss 0.0512596070766449 4000 model_31 model saved
model_32 is training for dataset of 0
Model for 32 loaded from saved state.
torch.Size([540, 60, 9]) torch.Size([61, 60, 9]) torch.Size([540, 60, 9]) torch.Size([61, 60, 9])
[74 26 17 33 17  1  0  0  0]
Training loss 0.10684186220169067 Testing loss 0.08818105608224869 0 model_32 model saved
[65 34  0 33 30 23  0  0  0]
Training loss 0.04800071939826012 Testing loss 0.0756874829530716 1000 model_32 model saved
[63 33  0 33 30 23  0  0  0]
Training loss 0.04516847804188728 Testing loss 0.07932552695274353 2

[98 62 12 38 33 21 18  0  0]
Training loss 0.030481837689876556 Testing loss 0.06397031247615814 2000 model_41 model saved
[98 61 12 38 32 21 18  0  0]
Training loss 0.029487930238246918 Testing loss 0.0639495849609375 3000 model_41 model saved
[98 61 12 38 32 21 18  0  0]
Training loss 0.028886055573821068 Testing loss 0.06424275040626526 4000 model_41 model saved
model_42 is training for dataset of 0
Model for 42 loaded from saved state.
torch.Size([540, 60, 9]) torch.Size([61, 60, 9]) torch.Size([540, 60, 9]) torch.Size([61, 60, 9])
[170 142  81  99  36  34  20   1   0]
Training loss 0.07296252250671387 Testing loss 0.06588174402713776 0 model_42 model saved
[192 164  55 103  83  55  29  13   0]
Training loss 0.01395464688539505 Testing loss 0.04924739897251129 1000 model_42 model saved
[189 164  55 102  84  54  29  13   0]
Training loss 0.00884715374559164 Testing loss 0.04616740718483925 2000 model_42 model saved
[188 165  56 100  86  54  29  14   0]
Training loss 0.00692242477089

...

[393 284  84 222 168  76  79  22   0]
Training loss 0.11788398027420044 Testing loss 0.09642855823040009 0 model_9 model saved
[354 232 153 180  81  77  60  19   0]
Training loss 0.00881887972354889 Testing loss 0.05778869241476059 1000 model_9 model saved
[352 226 153 177  80  77  59  19   0]
Training loss 0.006022571120411158 Testing loss 0.0536530427634716 2000 model_9 model saved
[353 224 153 176  81  77  59  19   0]
Training loss 0.004729794338345528 Testing loss 0.05111796408891678 3000 model_9 model saved
[354 224 153 176  82  77  59  19   0]
Training loss 0.003918189089745283 Testing loss 0.04968380182981491 4000 model_9 model saved
model_10 is training for dataset of 1
Model for 10 loaded from saved state.
torch.Size([540, 60, 9]) torch.Size([61, 60, 9]) torch.Size([540, 60, 9]) torch.Size([61, 60, 9])
[199 133  36 100  88  30  27   0   0]
Training loss 0.12635290622711182 Testing loss 0.08340492099523544 0 model_10 model saved
[181 116  82  83  39  29  19   0   0]
Training lo

[119  87  41  64  25  28  23  -1   0]
Training loss 0.019544580951333046 Testing loss 0.03783264011144638 1000 model_19 model saved
[117  88  41  65  25  26  23   0   0]
Training loss 0.01846501976251602 Testing loss 0.03918066993355751 2000 model_19 model saved
[115  88  41  64  24  26  23   0   0]
Training loss 0.017992017790675163 Testing loss 0.03980502486228943 3000 model_19 model saved
[114  89  42  64  23  25  23   0   0]
Training loss 0.01769992709159851 Testing loss 0.040238481014966965 4000 model_19 model saved
model_20 is training for dataset of 1
Model for 20 loaded from saved state.
torch.Size([540, 60, 9]) torch.Size([61, 60, 9]) torch.Size([540, 60, 9]) torch.Size([61, 60, 9])
[26  0  0  0  0  0  0  0  0]
Training loss 0.009628934785723686 Testing loss 0.006157093681395054 0 model_20 model saved
[25  0  0  0  0  0  0  0  0]
Training loss 0.009487464092671871 Testing loss 0.0061452388763427734 1000 model_20 model saved
[25  0  0  0  0  0  0  0  0]
Training loss 0.00948917

[124  49  45  47  29  20   0   0   0]
Training loss 0.017365941777825356 Testing loss 0.03866461664438248 3000 model_29 model saved
[124  49  45  48  29  19   0   0   0]
Training loss 0.016637878492474556 Testing loss 0.039114922285079956 4000 model_29 model saved
model_30 is training for dataset of 1
Model for 30 loaded from saved state.
torch.Size([540, 60, 9]) torch.Size([61, 60, 9]) torch.Size([540, 60, 9]) torch.Size([61, 60, 9])
[203  89  31  74  99  43  29   0   0]
Training loss 0.13149750232696533 Testing loss 0.07221511006355286 0 model_30 model saved
[165  58  54  83  41  26  10   0   0]
Training loss 0.013228820636868477 Testing loss 0.05364280939102173 1000 model_30 model saved
[165  58  54  82  42  24  10   0   0]
Training loss 0.0105519350618124 Testing loss 0.05406089127063751 2000 model_30 model saved
[162  59  53  80  42  23  10   0   0]
Training loss 0.009298348799347878 Testing loss 0.05402549356222153 3000 model_30 model saved
[163  59  53  80  43  22  10   0   0]
T

[84 50 29 35 18 17  0  0  0]
Training loss 0.024566898122429848 Testing loss 0.049831561744213104 4000 model_39 model saved
model_40 is training for dataset of 1
Model for 40 loaded from saved state.
torch.Size([540, 60, 9]) torch.Size([61, 60, 9]) torch.Size([540, 60, 9]) torch.Size([61, 60, 9])
[206 160  51  97  98  55  30  11   0]
Training loss 0.11656951904296875 Testing loss 0.06941759586334229 0 model_40 model saved
[253 184 101 129  85  60  29  18   0]
Training loss 0.008156612515449524 Testing loss 0.04712992161512375 1000 model_40 model saved
[251 183 101 130  81  58  28  17   0]
Training loss 0.0059544239193201065 Testing loss 0.044677965342998505 2000 model_40 model saved
[249 181 101 128  80  58  28  16   0]
Training loss 0.004877700470387936 Testing loss 0.043364547193050385 3000 model_40 model saved
[247 178 100 126  79  57  28  15   0]
Training loss 0.004146382212638855 Testing loss 0.04250069335103035 4000 model_40 model saved
model_41 is training for dataset of 1
Model

model_50 is training for dataset of 1
Model for 50 loaded from saved state.
torch.Size([540, 60, 9]) torch.Size([61, 60, 9]) torch.Size([540, 60, 9]) torch.Size([61, 60, 9])
[33 20 13 18 11  0  0  0  0]
Training loss 0.03772486373782158 Testing loss 0.018544543534517288 0 model_50 model saved
[40 18 19 16  0  0  0  0  0]
Training loss 0.02755897305905819 Testing loss 0.01601320505142212 1000 model_50 model saved
[39 18 19 16  0  0  0  0  0]
Training loss 0.027491312474012375 Testing loss 0.016142701730132103 2000 model_50 model saved
[40 19 20 16  0  0  0  0  0]
Training loss 0.027451252564787865 Testing loss 0.01644742488861084 3000 model_50 model saved
[40 18 20 16  0  0  0  0  0]
Training loss 0.027442317456007004 Testing loss 0.01639304682612419 4000 model_50 model saved
model_51 is training for dataset of 1
Model for 51 loaded from saved state.
torch.Size([540, 60, 9]) torch.Size([61, 60, 9]) torch.Size([540, 60, 9]) torch.Size([61, 60, 9])
[549 559 257 321 290 134  80  37   0]
Tr